In [ ]:
import os

import numpy as np
import torch
from lib.config import cfg

import lib.dataset.dataset3D as dataUtils
from lib.hybridnet.hybridnet import HybridNet
from lib.config.project_manager import ProjectManager

project = ProjectManager()
project.load('Coefficient_0 (copy)')

#project.create_new(
#    name = 'CSV Creation',
#    dataset2D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen',
#    dataset3D_path = '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen')

cfg = project.get_cfg()

In [ ]:
training_set = dataUtils.Dataset3D(cfg, set='train')
val_set = dataUtils.Dataset3D(cfg, set='val')
calibPaths = [training_set.coco.dataset['calibration']['intrinsics'], 
              training_set.coco.dataset['calibration']['extrinsics']]
print (calibPaths[0])

In [ ]:
hybridNet = HybridNet('inference', cfg, calibPaths, '/home/trackingsetup/Documents/Vortex/projects/Coefficient_0/models/vortex/Run_20211007-175714/Vortex-d_20.pth')

In [ ]:
from lib.hybridnet.modules.efficientdet.efficientdet import EfficientDet 
efficientDet = EfficientDet('inference', cfg, '/home/trackingsetup/Documents/Vortex/projects/Coefficient_0/models/efficientdet/Run_20211009-152053/efficientdet-d0_33.pth')

In [ ]:
import cv2
import matplotlib.pyplot as plt
img_paths = [
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_T/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RFT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RBT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LBT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LFT/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RC/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LC/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RFB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_RBB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LBB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_LFB/Frame_10274.jpg',
    '/home/trackingsetup/Documents/CombinedDataset/DatasetColleen/val/Colleen_03032021_Cara/Object_2/Camera_B/Frame_10274.jpg',
]
centerHMs = []
imgs = []
for img_path in img_paths:
    out_dict = efficientDet.predict_on_image(img_path)
    roi = out_dict['predictions'][0]['rois'][0]
    centerHM = np.array([int((roi[0]+roi[2])/2), int((roi[1]+roi[3])/2)])
    centerHMs.append(centerHM)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.
    img = (img - cfg.DATASET.MEAN) / cfg.DATASET.STD
    bbox_hw = 192
    img = img[centerHM[1]-bbox_hw:centerHM[1]+bbox_hw, centerHM[0]-bbox_hw:centerHM[0]+bbox_hw, :]
    imgs.append(img)

imgs = torch.from_numpy(np.array(imgs))
imgs = imgs.permute(0,3,1,2).view(1,12,3,384,384).cuda().float()
centerHMs = np.array(centerHMs)

center3D = torch.from_numpy(val_set.reproTool.reconstructPoint(centerHMs.transpose())).cuda()
center3D = center3D.int()
centerHMs = torch.from_numpy(centerHMs).cuda()

heatmap3D, heatmaps_padded, points3D_net = hybridNet.model(imgs, torch.unsqueeze(centerHMs,0), torch.unsqueeze(center3D, 0))
points2D = []
for point in points3D_net[0].cpu().numpy():
    points2D.append(val_set.reproTool.reprojectPoint(point))

img = cv2.cvtColor(cv2.imread(img_paths[11]), cv2.COLOR_BGR2RGB)
for point in points2D:
    cv2.circle(img, (int(point[11][0]), int(point[11][1])), 2, (255,0,0), thickness=5)
plt.imshow(img)
plt.imsave('test.png',img)
#for img_path in img_paths:
#    out_dict = efficientDet.predict_on_image(img_path)
#    print (out_dice)
#    efficientDet.visualize_prediction(out_dict)

In [ ]:
import cv2
import csv
import matplotlib.pyplot as plt
import itertools
from joblib import Parallel, delayed

    

video_paths = [
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_B.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_LBB.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_LBT.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_LC.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_LFB.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_LFT.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_RBB.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_RBT.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_RC.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_RFB.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_RFT.avi',
    '/mnt/RecordingRaid/Recordings/12_Camera_Recordings/Recording_Matthias_18062021/Camera_T.avi', 
]


caps = []
outs = []
for path in video_paths:
    caps.append(cv2.VideoCapture(path))
    outs.append(cv2.VideoWriter(os.path.join('/home/trackingsetup/Documents/Vortex/Videos_Matthias', path.split('/')[-1]), cv2.VideoWriter_fourcc('M','J', 'P', 'G'), 100, (1280,1024)))

for cap in caps:
        cap.set(1,0) #99640
        
def process(cap):
    ret, img = cap.read()
    return img
        
counter = 0
    
with open('3D Data/matthias.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                                quotechar='"', quoting=csv.QUOTE_MINIMAL)
    header = ['Frame Idx']
    joints = ["Pinky_T","Pinky_D","Pinky_M","Pinky_P","Ring_T","Ring_D","Ring_M","Ring_P","Middle_T","Middle_D","Middle_M","Middle_P","Index_T","Index_D","Index_M","Index_P","Thumb_T","Thumb_D","Thumb_M","Thumb_P","Palm", "Wrist_U","Wrist_R"]
    joints = joints
    joints = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in joints))
    header = header + joints
    header2 = [""]
    header2 = header2 + ['x','y', 'z']*23
    print (header2)
    writer.writerow(header)
    writer.writerow(header2)

    ret = True
    while ret:
        if counter % 20 == 0:
            print (counter)
        counter += 1
        imgs = []
        imgs_orig = []
        centerHMs = []
        camsToUse = []
        
        imgs_orig = Parallel(n_jobs=12, require='sharedmem')(delayed(process)(cap) for cap in caps)
        out_dict = efficientDet.predict_on_image(None, imgs_orig)
        for i in range(12): 
            try:
                roi = out_dict['predictions'][i]['rois'][0]
                if out_dict['predictions'][i]['scores'][0] > 0.6:
                    prev_roi = roi
                    camsToUse.append(i)
            except: 
                roi = [0,0,0,0]
            centerHM = np.array([max(min(int((roi[0]+roi[2])/2),1280-192), 192), 
                                 max(min(int((roi[1]+roi[3])/2), 1024-192),192)])
            centerHMs.append(centerHM)
            img = cv2.cvtColor(imgs_orig[i], cv2.COLOR_BGR2RGB)
            img = img.astype(np.float32) / 255.
            img = (img - cfg.DATASET.MEAN) / cfg.DATASET.STD
            bbox_hw = 192
            img = img[centerHM[1]-bbox_hw:centerHM[1]+bbox_hw, centerHM[0]-bbox_hw:centerHM[0]+bbox_hw, :]
            imgs.append(img)

        if not ret:
            break
        imgs = torch.from_numpy(np.array(imgs))
        imgs = imgs.permute(0,3,1,2).view(1,12,3,384,384).cuda().float()
        centerHMs = np.array(centerHMs)
        if len(camsToUse) >= 2:
            center3D = torch.from_numpy(val_set.reproTool.reconstructPoint(centerHMs.transpose(), camsToUse)).cuda()
            center3D = center3D.int()
            centerHMs = torch.from_numpy(centerHMs).cuda()

            heatmap3D, heatmaps_padded, points3D_net = hybridNet.model(imgs, torch.unsqueeze(centerHMs,0), torch.unsqueeze(center3D, 0))
            row = [counter]
            for point in points3D_net.squeeze():
                row = row + point.tolist()
            writer.writerow(row)
            points2D = []
            for point in points3D_net[0].cpu().numpy():
                points2D.append(val_set.reproTool.reprojectPoint(point))

            colors = [(255,0,0), (255,0,0),(255,0,0),(255,0,0),(0,255,0),(0,255,0),(0,255,0),(0,255,0),(0,0,255),(0,0,255),(0,0,255),(0,0,255),(255,255,0),(255,255,0),(255,255,0), (255,255,0),
                      (0,255,255),(0,255,255),(0,255,255),(0,255,255), (255,0,255),(100,0,100),(100,0,100)]

        else:
            row = [counter]
            for i in range(23*3):
                row = row + ['NaN']
            writer.writerow(row)
            
        for i,out in enumerate(outs):
            img = imgs_orig[i]
            if len(camsToUse) >= 2:
                for j,point in enumerate(points2D):
                    cv2.circle(img, (int(point[i][0]), int(point[i][1])), 2, colors[j], thickness=5)
            out.write(img)
            

    

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

figure = plt.figure()
axes = figure.gca(projection='3d')

c = ['r', 'r','r','r','b','b','b','b','g','g','g','g', 'orange', 'orange','orange','orange', 'y','y','y','y','purple', 'purple','purple']
line_idxs = [[0,1], [1,2], [2,3], [4,5], [5,6], [6,7], [8,9], [9,10], [10,11], [12,13], [13,14], [14,15], [16,17], [17,18], [18,19], [3,7], [7,11], [11,15], [3,21], [7,21],[11,22], [15,22],[21,22], [18,15], [19,22]]
points3D = points3D_net[0].cpu().numpy()
center3D = center3D.cpu()
for i, point in enumerate(points3D):
    #print ("Classic:", i, point)
    if i != 20:
        axes.scatter(point[0], point[1], point[2], color = c[i])
for line in line_idxs:
    axes.plot([points3D[line[0]][0], points3D[line[1]][0]], [points3D[line[0]][1], points3D[line[1]][1]], [points3D[line[0]][2], points3D[line[1]][2]], c = 'gray')

axes.set_xlim3d(center3D[0]-100, center3D[0]+100)
axes.set_ylim3d(center3D[1]-100, center3D[1]+100)
axes.set_zlim3d(center3D[2]-100, center3D[2]+100)
plt.subplots_adjust(left=0., right=1., top=1., bottom=0.)
#plt.gca().invert_zaxis()
plt.savefig('test.png')
plt.show()